In [12]:
from zoneinfo import ZoneInfo
import datetime as dt
import os
import asyncio

import numpy as np
import pandas as pd

import utils
from weather_sensor_reading import sensor_reading_history
import weather_info
%load_ext autoreload
%autoreload 2
%autoawait

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
IPython autoawait is `on`, and set to use `asyncio`


In [13]:
k_site_day_mkt2price = {}
k_site2days = utils.get_events_hardcoded()

for k in k_site2days:
    print(k)
    for evt in k_site2days[k]:
        print(" "* 2, evt)
        mkts = utils.get_markets(evt,k)
        for mkt in mkts:
            print(" " * 4, mkt)


NY
   KXHIGHNY-25MAY23
     KXHIGHNY-25MAY23-T56
     KXHIGHNY-25MAY23-B56.5
     KXHIGHNY-25MAY23-B58.5
     KXHIGHNY-25MAY23-B60.5
     KXHIGHNY-25MAY23-B62.5
     KXHIGHNY-25MAY23-T63
   KXHIGHNY-25MAY24
     KXHIGHNY-25MAY24-T62
     KXHIGHNY-25MAY24-B62.5
     KXHIGHNY-25MAY24-B64.5
     KXHIGHNY-25MAY24-B66.5
     KXHIGHNY-25MAY24-B68.5
     KXHIGHNY-25MAY24-T69
CHI
   KXHIGHCHI-25MAY23
     KXHIGHCHI-25MAY23-T59
     KXHIGHCHI-25MAY23-B59.5
     KXHIGHCHI-25MAY23-B61.5
     KXHIGHCHI-25MAY23-B63.5
     KXHIGHCHI-25MAY23-B65.5
     KXHIGHCHI-25MAY23-T66
   KXHIGHCHI-25MAY24
     KXHIGHCHI-25MAY24-T60
     KXHIGHCHI-25MAY24-B60.5
     KXHIGHCHI-25MAY24-B62.5
     KXHIGHCHI-25MAY24-B64.5
     KXHIGHCHI-25MAY24-B66.5
     KXHIGHCHI-25MAY24-T67
MIA
   KXHIGHMIA-25MAY23
     KXHIGHMIA-25MAY23-T86
     KXHIGHMIA-25MAY23-B86.5
     KXHIGHMIA-25MAY23-B88.5
     KXHIGHMIA-25MAY23-B90.5
     KXHIGHMIA-25MAY23-B92.5
     KXHIGHMIA-25MAY23-T93
   KXHIGHMIA-25MAY24
     KXHIGHMIA-25MAY24-T84


In [14]:
for mkt_site in weather_info.kalshi_sites:
    nws_site = weather_info.kalshi_site2nws_site[mkt_site]
    tz = weather_info.nws_site2tz[nws_site]
    snsrReading = sensor_reading_history(nws_site)
    fdir = f'sensors/{nws_site}'
    os.makedirs(fdir, exist_ok=True)
    fpath = f'{fdir}/sensorData.csv'
    if os.path.exists(fpath):
        snsrHist = pd.read_csv(fpath, index_col=0, parse_dates=True)
        snsrHist.index = pd.to_datetime(snsrHist.index, utc=True).tz_convert(tz)
        snsrReading.index = snsrReading.index.tz_convert(tz)
        new_readings = snsrReading[~snsrReading.index.isin(snsrHist.index)]
        
        if not new_readings.empty:
            snsrReading = pd.concat([snsrHist, new_readings])
    else:
        if not snsrReading.empty:
            snsrReading.index = snsrReading.index.tz_convert(tz)
        else: 
            print(f"No sensor readings available for {nws_site}")
            continue
    snsrReading.index = snsrReading.index.tz_localize(None)
    snsrReading.to_csv(fpath)